# MMA Bootcamp - Intro to SQL

## 1. Preparation

We start by setting up a connection to our sample database, Northwind DB.

In [39]:
# download Northwind SQLite DB
!wget https://tdmdal.github.io/mma-sql/data/northwind.sqlite3

--2023-10-11 03:37:12--  https://tdmdal.github.io/mma-sql/data/northwind.sqlite3
Resolving tdmdal.github.io (tdmdal.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to tdmdal.github.io (tdmdal.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 804864 (786K) [application/octet-stream]
Saving to: ‘northwind.sqlite3.2’

northwind.sqlite3.2 100%[===================>] 786.00K  --.-KB/s    in 0.06s   

2023-10-11 03:37:13 (12.2 MB/s) - ‘northwind.sqlite3.2’ saved [804864/804864]



In [40]:
# load the SQL magic extension
# https://github.com/catherinedevlin/ipython-sql
# this extension allows us to connect to DBs and issue SQL command
%load_ext sql

# now we can use the magic extension to connect to our SQLite DB
# use %sql to write an inline SQL command
# use %%sql to write SQL commands in a cell
%sql sqlite:///northwind.sqlite3

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


After connecting to the Northwind DB, let's first check what database engine we are using.

In [41]:
%%sql
SELECT sqlite_version();

 * sqlite:///northwind.sqlite3
Done.


sqlite_version()
3.37.2


We are using [SQLite 3](https://www.sqlite.org/index.html). It's a small and fast Relational Database engine. SQLite DB is self-contained: one database is just a file. Believe it or not, SQLite is the most used DB engine in the world. It's built into millions of mobile phones (e.g. Android or iOS phones) and it's used by many popular apps to store data.  

## 2. A quick look at Northwind DB

### How many tables does this DB have?

A database may contain many tables. Let's see how many tables we have in the Northwind DB.

Every SQLite database has a special table called `sqlite_master`. It contains a master listing of all database objects (tables, indexes, etc.) in the database and the SQL used to create each object. We can query this table to find out how many tables (excluding `sqlite_master`) we have in our Northwind DB.

We will use SQL keywords `SELECT...FROM...WHERE`. Don't worry about them for now. We will soon learn those keywords.

In [42]:
%%sql
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';

 * sqlite:///northwind.sqlite3
Done.


name
Suppliers
Shippers
Employees
Customers
Orders
Categories
Products
OrderDetails


Note that special tables in SQLite start with `sqlite_`. Those are reserved tables for SQLite engine system use. Above we filtered out those tables. In most cases, we shouldn't touch those special tables.

### How does the Categories table look like?

A table is made up of one or more columns/fields. Let's take a look a the Categories table (column names, types, etc.).

In [43]:
%sql PRAGMA table_info([Categories]);

 * sqlite:///northwind.sqlite3
Done.


cid,name,type,notnull,dflt_value,pk
0,CategoryID,INTEGER,1,None,1
1,CategoryName,nvarchar(15),1,None,0
2,Description,ntext,0,None,0
3,Picture,image,0,None,0


### How about the relationships between the tables?

We present the relationships between the tables using an Entity Relationship (ER) diagram.

<img src="https://tdmdal.github.io/mma-sql/images/NW_ER.png" alt="ER Diagram" width="836" height="605" />

In the above ER diagram, the tiny vertical key icon indicates a column is a primary key. A primary key is a column (or set of columns) whose values uniquely identify every row in a table. For example, `OrderID` is the primary key in the `Orders` table, and `OrderID` and `ProductID` (combined) is the primary key in the `OrderDetails` table.

The relationship icon (a line with a horizontal key at one end and an infinite symbol at the other end) indicates a foreign key constraint and a one-to-many relationship. A foreign key is a column (or set of columns) in one table whose values uniquely identify a row of another table or the same table. A foreign key mostly refers to a primary key in another table. A foreign key constraint requires that the constrained column contain only values from the primary key column of the other table. For example `CustomerID` in the `Orders` table is a foreign key that refers to the `CustomerID` primary key in the `Customers` table, and it can only contain values that exist in the `CustomerID` column of the `Customers` table.

In addition, it happens that every foreign key constraint in the Northwind DB establishes a one-to-many relationship, i.e. a row from one table can have multiple matching rows in another table. For example, one row from the `Customers` table can match multiple rows in the `Orders` table (via `CustomerID`). This makes sense as one customer can place more than one orders. (Another common relationship a foreign key constraint can establish is the one-to-one relationship.)

|logo|meaning|
|:------:|:------:|
|![key logo](https://tdmdal.github.io/mma-sql/images/key_vertical.png "key logo")|primary key|
|![foreign key constraint](https://tdmdal.github.io/mma-sql/images/relationship.png "foreign key constraint")|one-to-many foreign key contraint|

**Question**: Why do we need foreign key constraints? (Discussion)

You can find out relationships between tables by querying the `sqlite_master` table. See the code below.

Of course, querying the `sqlite_master` table to find out relationships between tables is specific to SQLite. We would do it differently if we use other database management systems. For example, if we use MySQL system, we would query the `key_column_usage` table in the `information_schema` DB. See this stackoverflow [question](https://stackoverflow.com/questions/20855065/how-to-find-all-the-relations-between-all-mysql-tables).

Alternatively, some SQL client tools can generate ER diagram for you. The above diagram is generated by SQL Server Management Studio (SSMS). The Northwind DB is a sample DB originally shipped with Microsoft SQL Server.

In [44]:
%%sql
-- find out relationships between tables using SQL command
SELECT sql
FROM sqlite_master
WHERE name = "Orders"

 * sqlite:///northwind.sqlite3
Done.


sql
"CREATE TABLE [Orders] ( [OrderID] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CustomerID] nchar(5) NOT NULL COLLATE NOCASE, [EmployeeID] int NULL, [OrderDate] datetime NULL, [RequiredDate] datetime NULL, [ShippedDate] datetime NULL, [ShipVia] int NULL, [Freight] money DEFAULT 0 NULL, [ShipName] nvarchar(40) NULL COLLATE NOCASE, [ShipAddress] nvarchar(60) NULL COLLATE NOCASE, [ShipCity] nvarchar(15) NULL COLLATE NOCASE, [ShipRegion] nvarchar(15) NULL COLLATE NOCASE, [ShipPostalCode] nvarchar(10) NULL COLLATE NOCASE, [ShipCountry] nvarchar(15) NULL COLLATE NOCASE, CONSTRAINT [FK_Orders_Customers] FOREIGN KEY ([CustomerID]) REFERENCES [Customers] ([CustomerID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Employees] FOREIGN KEY ([EmployeeID]) REFERENCES [Employees] ([EmployeeID]) ON DELETE NO ACTION ON UPDATE NO ACTION, CONSTRAINT [FK_Orders_Shippers] FOREIGN KEY ([ShipVia]) REFERENCES [Shippers] ([ShipperID]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


## Part 3. Join Tables

1. Inner join: `SELECT...FROM...INNER JOIN...ON...`
2. Left join:  `SELECT...FROM...LEFT JOIN...ON...`
3. Other joins.

### Ex.16* Show products and their associated suppliers

Display the `ProductID`, `ProductName`, and the `CompanyName` of the Supplier. Sort by `ProductID`.

We can use the `WHERE` keyword.

In [45]:
%%sql
-- start your code here
SELECT Products.ProductID, Products.ProductName, Suppliers.CompanyName
FROM Products, Suppliers
WHERE Products.SupplierID = Suppliers.SupplierID
ORDER BY Products.ProductID;

 * sqlite:///northwind.sqlite3
Done.


ProductID,ProductName,CompanyName
1,Chai,Exotic Liquids
2,Chang,Exotic Liquids
3,Aniseed Syrup,Exotic Liquids
4,Chef Anton's Cajun Seasoning,New Orleans Cajun Delights
5,Chef Anton's Gumbo Mix,New Orleans Cajun Delights
6,Grandma's Boysenberry Spread,Grandma Kelly's Homestead
7,Uncle Bob's Organic Dried Pears,Grandma Kelly's Homestead
8,Northwoods Cranberry Sauce,Grandma Kelly's Homestead
9,Mishi Kobe Niku,Tokyo Traders
10,Ikura,Tokyo Traders


The above SQL code retrieves product information along with the corresponding supplier company names, joining the "Products" and "Suppliers" tables on the "SupplierID" column and sorting the results by product ID.

We can also use the `(INNER) JOIN` keyword.

In [46]:
%%sql
-- start your code here
SELECT Products.ProductID, Products.ProductName, Suppliers.CompanyName
FROM Products
INNER JOIN Suppliers ON Products.SupplierID = Suppliers.SupplierID
ORDER BY Products.ProductID;

 * sqlite:///northwind.sqlite3
Done.


ProductID,ProductName,CompanyName
1,Chai,Exotic Liquids
2,Chang,Exotic Liquids
3,Aniseed Syrup,Exotic Liquids
4,Chef Anton's Cajun Seasoning,New Orleans Cajun Delights
5,Chef Anton's Gumbo Mix,New Orleans Cajun Delights
6,Grandma's Boysenberry Spread,Grandma Kelly's Homestead
7,Uncle Bob's Organic Dried Pears,Grandma Kelly's Homestead
8,Northwoods Cranberry Sauce,Grandma Kelly's Homestead
9,Mishi Kobe Niku,Tokyo Traders
10,Ikura,Tokyo Traders


The above SQL code achieves the same result as the previous one but uses the "INNER JOIN" clause for better readability and explicitness, retrieving product information and corresponding supplier company names while sorting the results by product ID.

### Ex.17 Show all orders and shippers with OrderID less than 10255.

Display `OrderID`, `OrderDate` (date only), and `CompanyName` of the Shipper, and sort by `OrderID`. In addition, only display rows with `OrderID < 10255`.

In [47]:
%%sql
SELECT Orders.OrderID, DATE(Orders.OrderDate) AS OrderDate, Shippers.CompanyName
FROM Orders
INNER JOIN Shippers ON Orders.ShipVia = Shippers.ShipperID
WHERE Orders.OrderID < 10255
ORDER BY Orders.OrderID;

 * sqlite:///northwind.sqlite3
Done.


OrderID,OrderDate,CompanyName
10248,2014-07-04,Federal Shipping
10249,2014-07-05,Speedy Express
10250,2014-07-08,United Package
10251,2014-07-08,Speedy Express
10252,2014-07-09,United Package
10253,2014-07-10,United Package
10254,2014-07-11,United Package


The code selects and formats order information, including order ID and date, along with the shipper's company name, for orders with an order ID less than 10255, and it sorts the results by order ID.

### Ex.18* Find total number of products in each category

In your result, display `CategoryName` and total number of product.

In [48]:
%%sql
-- start your code here
SELECT Categories.CategoryName, COUNT(Products.ProductID) AS TotalProducts
FROM Categories
LEFT JOIN Products ON Categories.CategoryID = Products.CategoryID
GROUP BY Categories.CategoryName;


 * sqlite:///northwind.sqlite3
Done.


CategoryName,TotalProducts
Beverages,12
Condiments,12
Confections,13
Dairy Products,10
Grains/Cereals,7
Meat/Poultry,6
Produce,5
Seafood,12


The SQL code retrieves the category names and the total count of products within each category, even if some categories have no products, using a left join, and groups the results by category name.

### Ex.19 Show all orders with values greater than $12,000 and are placed in 2016?

In your result, display `OrderID` and total value of the order (ignore `Discount`).

In [49]:
%%sql
-- start your code SELECT OrderID, (UnitPrice * Quantity) AS TotalValue
SELECT OrderDetails.OrderID, SUM(UnitPrice * Quantity) AS TotalValue
FROM OrderDetails
JOIN Orders ON OrderDetails.OrderID = Orders.OrderID
WHERE strftime('%Y', OrderDate) = '2016'
GROUP BY OrderDetails.OrderID
HAVING TotalValue > 12000;

 * sqlite:///northwind.sqlite3
Done.


OrderID,TotalValue
10865,17250.0
10981,15810.0
11030,16321.900000000001


The above code calculates the total value of orders placed in the year 2016 by multiplying the unit price by the quantity for each order detail, and then sums these values for each order. It selects only orders with a total value greater than $12,000 and groups the results by order ID.

### Ex.20* What products has Michael Suyama ever sold?

In your result, display `EmployeeID`, `FirstName`, `LastName`, `ProductID` and `ProductName`.

In [50]:
%%sql
-- start your code here
SELECT E.EmployeeID, E.FirstName, E.LastName, OD.ProductID, P.ProductName
FROM Employees AS E
JOIN Orders AS O ON E.EmployeeID = O.EmployeeID
JOIN OrderDetails AS OD ON O.OrderID = OD.OrderID
JOIN Products AS P ON OD.ProductID = P.ProductID
WHERE E.FirstName = 'Michael' AND E.LastName = 'Suyama'
Group BY P.ProductID;


 * sqlite:///northwind.sqlite3
Done.


EmployeeID,FirstName,LastName,ProductID,ProductName
6,Michael,Suyama,1,Chai
6,Michael,Suyama,2,Chang
6,Michael,Suyama,3,Aniseed Syrup
6,Michael,Suyama,4,Chef Anton's Cajun Seasoning
6,Michael,Suyama,5,Chef Anton's Gumbo Mix
6,Michael,Suyama,7,Uncle Bob's Organic Dried Pears
6,Michael,Suyama,10,Ikura
6,Michael,Suyama,11,Queso Cabrales
6,Michael,Suyama,12,Queso Manchego La Pastora
6,Michael,Suyama,13,Konbu


The code retrieves information about products ordered by the employee named Michael Suyama. It lists the employee's ID, first name, and last name along with the product ID and product name for the products ordered by this employee, grouping the results by product ID.

### Ex.21* Find customers that never placed an order

In [51]:
%%sql
-- start your code here
SELECT Customers.CustomerID, Customers.ContactName, Customers.CompanyName
FROM Customers
LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID
WHERE Orders.CustomerID IS NULL;


 * sqlite:///northwind.sqlite3
Done.


CustomerID,ContactName,CompanyName
FISSA,Diego Roel,FISSA Fabrica Inter. Salchichas S.A.
PARIS,Marie Bertrand,Paris spécialités


The code retrieves customer information (customer ID, contact name, and company name) for customers who have not placed any orders. It does so by using a left join between the "Customers" and "Orders" tables and filtering for cases where the customer ID in the "Orders" table is null, indicating no orders have been placed by that customer.

Note that the Jupyter Notebook displays Null as None.



### Ex.22* Find customers who never placed an order from Margaret Peacock (EmployeeID 4)

In [52]:
%%sql
-- start your code here
SELECT Customers.CustomerID, Customers.ContactName
FROM Customers
LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID
LEFT JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
WHERE Employees.EmployeeID != 4 OR Employees.EmployeeID IS NULL
Group by Customers.CustomerID,Customers.ContactName;

 * sqlite:///northwind.sqlite3
Done.


CustomerID,ContactName
ALFKI,Maria Anders
ANATR,Ana Trujillo
ANTON,Antonio Moreno
AROUT,Thomas Hardy
BERGS,Christina Berglund
BLAUS,Hanna Moos
BLONP,Frédérique Citeaux
BOLID,Martín Sommer
BONAP,Laurence Lebihan
BOTTM,Elizabeth Lincoln


The SQL code retrieves customer information (customer ID and contact name) for customers who were either not served by employee ID 4 or have never placed an order. It uses left joins to connect the "Customers," "Orders," and "Employees" tables, and then it filters for cases where the employee ID is not equal to 4 or is null (indicating no orders have been placed by that customer). The results are grouped by customer ID and contact name.

### Ex.23* Find high-value customers - 1

We define high-value customers as those who have made at least 1 order with a total value >= $10,000 (ignore discount). We only consider orders placed in year 2016.

In [53]:
%%sql
-- start your code here
SELECT C.CustomerID, C.ContactName, SUM(OD.UnitPrice * OD.Quantity) AS TotalOrderValue
FROM Customers AS C
JOIN Orders AS O ON C.CustomerID = O.CustomerID
JOIN OrderDetails AS OD ON O.OrderID = OD.OrderID
WHERE strftime('%Y', O.OrderDate) = '2016'
GROUP BY C.CustomerID, C.ContactName
HAVING TotalOrderValue >= 10000;

 * sqlite:///northwind.sqlite3
Done.


CustomerID,ContactName,TotalOrderValue
BOTTM,Elizabeth Lincoln,12227.4
ERNSH,Roland Mendel,42598.9
FOLKO,Maria Larsson,15973.85
GREAL,Howard Snyder,10562.58
HANAR,Mario Pontes,24238.05
HUNGO,Patricia McKenna,22796.340000000004
KOENE,Philip Cramer,20204.95
LINOD,Felipe Izquierdo,10085.6
QUICK,Horst Kloss,40526.99
RATTC,Paula Wilson,21725.600000000002


The SQL code calculates the total order value for customers' orders placed in the year 2016. It selects customer information (customer ID and contact name) and sums the product of unit price and quantity from order details to calculate the total order value. It then filters the results to include only customers whose total order value is greater than or equal to $10,000 and groups the results by customer ID and contact name.

### Ex.24 Find high-value customers - 2

This time, we define high-value customers as those who have made total order >= $15,000 (ignore discount). We only consider orders placed in year 2016.

In [54]:
%%sql
-- start your code here
SELECT C.CustomerID, C.ContactName,SUM(OD.UnitPrice * OD.Quantity) AS TotalOrderValue
FROM Customers AS C
JOIN Orders AS O ON C.CustomerID = O.CustomerID
JOIN OrderDetails AS OD ON O.OrderID = OD.OrderID
WHERE strftime('%Y', O.OrderDate) = '2016'
GROUP BY C.CustomerID, C.ContactName
HAVING TotalOrderValue >= 15000;

 * sqlite:///northwind.sqlite3
Done.


CustomerID,ContactName,TotalOrderValue
ERNSH,Roland Mendel,42598.9
FOLKO,Maria Larsson,15973.85
HANAR,Mario Pontes,24238.05
HUNGO,Patricia McKenna,22796.340000000004
KOENE,Philip Cramer,20204.95
QUICK,Horst Kloss,40526.99
RATTC,Paula Wilson,21725.600000000002
SAVEA,Jose Pavarotti,42806.25
WHITC,Karl Jablonski,15278.9


The SQL code calculates the total order value for customers' orders placed in the year 2016. It selects customer information (customer ID and contact name) and sums the product of unit price and quantity from order details to calculate the total order value. It then filters the results to include only customers whose total order value is greater than or equal to $15,000 and groups the results by customer ID and contact name.